In [37]:
%matplotlib inline 

import numpy as np
import math
import matplotlib.pyplot as plt

In [38]:
Xant1 = 434287492
Xant2 = 514324426
res = []
tamXi = 10000
M = 99999.0
T = 40.0
at = []
an = []
LLt = []
St = []

In [39]:
def gencongru1(): 
    global Xant1
    a = 134775813
    c = 1
    m = 2**32
    Xnext = (a * Xant1 + c) % m
    Xant1 = Xnext
    U = Xnext / m
    return U

def gencongru2():
    global Xant2
    a = 1140671485
    c = 12820163
    m = 2**24
    Xnext = (a * Xant2 + c) % m
    Xant2 = Xnext
    U = Xnext / m
    return U

def GenerarX():
    vlambda = 3
    U = gencongru1()
    X = -(math.log(1-U))/float(vlambda)
    return X

def GenerarY():
    vlambda = 7
    U = gencongru2()
    Y = -(math.log(1-U))/float(vlambda)
    return Y

def Llegada(tsuc):
    global n, NLL, NS, t, S, Serv, LL, at, an
    t=tsuc
    n = n + 1
    LLt.append(t)
    at.append(t)
    an.append(n)
    
    NLL = NLL + 1
    LL.append(t)
    
    X = GenerarX()
    
    if (t+X) < T:
        TSuc['tLL'] = t + X
    if n==1:
        Y = GenerarY()
        TSuc['tS'] = t + Y
        #Serv[NS]=Y 
        Serv.append(Y)
        
def Servidor(tsuc):
    global n, NLL, NS, t, S, Serv, LL, at, an
    t = tsuc
    n = n - 1
    
    St.append(t)
    at.append(t)
    an.append(n)
    
    NS = NS + 1
    S.append(t)

    if n>0:
        Y = GenerarY()
        TSuc['tS'] = t + Y
        Serv.append(Y) 

In [50]:
for i in range(tamXi):
  M = 99999.0
  T = 40.0
  at = []
  an = []
  LLt = []
  St = []

  t = tsuc = NLL = NS = n = 0

  at.append(t)
  an.append(n)

  TSuc = {"tLL":M, "tS":M}

  LL = []
  S = []
  Serv = []
  LL.append(0.0)
  S.append(0.0)
  Serv.append(0.0)

  X = GenerarX()

  if X > T:
      Tp = t_med_sistema = t_med_cola = 0.0
      exit
  else:
      Llegada(X)
      while((TSuc['tLL'] != M)|(TSuc['tS']!=M)):
          if TSuc['tLL'] < TSuc['tS']:
              tsuc = TSuc['tLL']
              TSuc['tLL'] = M
              Llegada(tsuc)
              
          if TSuc['tS'] < TSuc['tLL']:
              tsuc = TSuc['tS']
              TSuc['tS'] = M
              Servidor(tsuc)

      Tp = max(0,t-T)
      acumulo1 = acumulo2 = 0.0
      ind = 0
      
      while ind < NLL:
          acumulo1 = acumulo1 + S[ind] - LL[ind]
          acumulo2 = acumulo2 + S[ind] - LL[ind] - Serv[ind]
          ind = ind + 1
      t_med_sistema = acumulo1/NLL
      t_med_cola = acumulo2/NLL

      exit
  an = np.array(an)
  res.append([t_med_sistema, t_med_cola, Tp, max(an), NLL])

In [42]:
#at = np.array(at)
#an = np.array(an)

#LLt = np.array(LLt)
#St = np.array(St)

#tmp = np.zeros((len(LLt)))+1

#plt.plot(LLt, tmp, 'r.')
#plt.xlim(0, max(at))
#plt.show()

#tmp = np.zeros((len(St)))+1

#plt.plot(St, tmp, 'b.')
#plt.xlim(0, max(at))
#plt.show()

#plt.plot(at,an)
#plt.xlabel('tiempo (t)')
#plt.ylabel('numero de clientes en el sistema (n)')
#plt.show()

#print("Tiempo medio de los clientes en el sistema: ",t_med_sistema)
#print("Tiempo medio de los clientes en la cola: ",t_med_cola)
#print("Tiempo transcurrido desde T hasta que el último cliente abandona el sistema: ",Tp)
#print("Número máximo de clientes en el sistema: ",max(an))
#print("Total de clientes que pasaron por el sistema: ",NLL)


**Laboratorio 3**

Laboratorio 3 -  Análisis Estadístico de Datos Simulados

Usando la implementación de ejemplo del sistema de línea de espera con un servidor (Modelo de colas G/G/1) en Notebook Jupyter, implementar:

La medida de desempeño de tiempo promedio en fila de los clientes (Xi) para una corrida de simulación i .

Implementar el método recursivo para calcular la media de la muestra y la varianza de la muestra.

Calcular la media de la muestra (sample mean) y la varianza de la muestra (sample variance) para:

a. k=100 simulaciones.

b. k=1000 simulaciones.

c. k=10000 simulaciones.

In [51]:

#sample Mean 
def sampleMean(tam, lista):
  listAux = []
  for i in range(tam):
    if i == 0 : 
      aux = lista[0][1]
    else:
      aux = listAux[i-1]+(lista[i][1]-listAux[i-1])/i
    
    listAux.append(aux)
  return listAux

#Sample Variance
def sampleVariance(tam, lista, listMean):
  listAux = []
  for i in range(tam):
    if i == 0:
      aux = 0
    else:
      aux = (1-(1/i-1))*listAux[i-1] + i * (listMean[i] - listMean[i-1])**2

    listAux.append(aux)
  return listAux

In [52]:
k = 100
listMean = sampleMean(k, res)
listVariance = sampleVariance(k, res, listMean)

print("K 100 --> Mean ",listMean[k-1]," variance ",listVariance[k-1])

k = 1000
listMean = sampleMean(k, res)
listVariance = sampleVariance(k, res, listMean)

print("K 1000 --> Mean ",listMean[k-1]," variance ",listVariance[k-1])

k = 10000
listMean = sampleMean(k, res)
listVariance = sampleVariance(k, res, listMean)

print("K 10000 --> Mean ",listMean[k-1]," variance ",listVariance[k-1])

K 100 --> Mean  0.0909291274614163  variance  9.211410132326329e+25
K 1000 --> Mean  0.09808975838292695  variance  2.4538673284968294e+296
K 10000 --> Mean  0.09947332058520446  variance  inf
